# Reproducibility & Continuous Integration

## Notebooks 

Notebooks offer a fairly self contained approach to reproducibility (see recent paper "The Scientific Paper Is Obsolete" https://www.theatlantic.com/science/archive/2018/04/the-scientific-paper-is-obsolete/556676/).

When the first gravitational wave detections were announced in 2016, Jupyter notebooks were front and center, allowing the public to (basically) reproduce the results from raw data.

https://losc.ligo.org/s/events/GW150914/LOSC_Event_tutorial_GW150914.html

## Binder 

Allowing people to not just download your notebooks, but also interact with them is a good step. 

Mybinder (https://mybinder.org/) is a hosted (via UC Berkeley) executation layer for your public notebooks.

<img src="imgs/binder.png">

You then add something like this to your README:

```markdown
[![Binder](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/profjsb/python-seminar/master)
```

Note that we often need to add dependencies (`environment.yml`)

In [ ]:
# %load https://raw.githubusercontent.com/profjsb/python-seminar/master/environment.yml
channels:
  - conda-forge
  - defaults
dependencies:
  - matplotlib
  - scipy
  - numpy
  - pandas
  - numexpr
  - bokeh
  - datashader
  - seaborn
  - pillow
  - ipywidgets
  - altair
  - plotly
  - bqplot
  - pip:
    - sphinx-gallery
    - pdvega
    - vega3



And execute a post build script (`postBuild`). Put these in your top level directory as needed.

In [ ]:
# %load https://raw.githubusercontent.com/profjsb/python-seminar/master/postBuild
#!/bin/bash
jupyter nbextension install --sys-prefix --py vega3
jupyter nbextension enable vega --py --sys-prefix
jupyter nbextension enable vega3 --py --sys-prefix


## Packaging and Testing

In [50]:
!git clone https://github.com/profjsb/PyAdder

Cloning into 'PyAdder'...
remote: Counting objects: 86, done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 86 (delta 9), reused 24 (delta 5), pack-reused 55
Unpacking objects: 100% (86/86), done.


In [ ]:
!brew install tree

In [51]:
!tree PyAdder/ -a -T PyAdder -C --noreport -I ".git|*.pyc"

PyAdder/
├── .coveragerc
├── .gitignore
├── .travis
│   └── run.sh
├── .travis.yml
├── CHANGES.txt
├── LICENSE.txt
├── MANIFEST.in
├── README.md
├── adder
│   ├── __init__.py
│   └── tests
│       ├── __init__.py
│       └── test_one_number.py
├── requirements.txt
├── setup.cfg
└── setup.py


See package management: https://packaging.python.org/ for more about the structure of this and the 02 lecture in `01_Versioning_Application_Building`. The meat of this python package is `adder/__init__.py`:

```python
__version__ = "0.0.3"
__author__ = "Josh!"

from numpy import array

def run(*args):
    return array(args).sum()
```


In [1]:
cd PyAdder/

/Users/jbloom/Classes/python-seminar/DataFiles_and_Notebooks/09_Web/PyAdder


In [2]:
!python setup.py sdist bdist_wheel

running sdist
running egg_info
writing PyAdder.egg-info/PKG-INFO
writing dependency_links to PyAdder.egg-info/dependency_links.txt
writing top-level names to PyAdder.egg-info/top_level.txt
reading manifest file 'PyAdder.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'PyAdder.egg-info/SOURCES.txt'
running check
creating PyAdder-0.0.3
creating PyAdder-0.0.3/PyAdder.egg-info
creating PyAdder-0.0.3/adder
creating PyAdder-0.0.3/adder/tests
copying files to PyAdder-0.0.3...
copying CHANGES.txt -> PyAdder-0.0.3
copying LICENSE.txt -> PyAdder-0.0.3
copying MANIFEST.in -> PyAdder-0.0.3
copying README.md -> PyAdder-0.0.3
copying requirements.txt -> PyAdder-0.0.3
copying setup.cfg -> PyAdder-0.0.3
copying setup.py -> PyAdder-0.0.3
copying PyAdder.egg-info/PKG-INFO -> PyAdder-0.0.3/PyAdder.egg-info
copying PyAdder.egg-info/SOURCES.txt -> PyAdder-0.0.3/PyAdder.egg-info
copying PyAdder.egg-info/dependency_links.txt -> PyAdder-0.0.3/PyAdder.egg-info
copying PyAdde

In [3]:
ls dist/

PyAdder-0.0.3-py3-none-any.whl  PyAdder-0.0.3.tar.gz


In [4]:
!pip install dist/PyAdder-0.0.3-py3-none-any.whl

In [6]:
import adder
print(f"version: {adder.__version__}")
adder.run(0x1, 233434 + 2j)

version: 0.0.3


(233435+2j)

In [ ]:
# %load adder/__init__.py
__version__ = "0.0.3"
__author__ = "Josh!"

from numpy import array

def run(*args):
    return array(args).sum()


In [22]:
%%writefile adder/__init__.py
__version__ = "0.0.4"
__author__ = "Josh!"

from numpy import array

def run(*args):
    return array(args).sum()

def run2(*args):
    return array(args).mean()


Overwriting adder/__init__.py


## Tests

We'd like to know if our code is working before we deploy. Locally we can run `py.test` or:

In [9]:
!rm -fr PyAdder

In [23]:
!python setup.py test

running pytest
running egg_info
writing PyAdder.egg-info/PKG-INFO
writing dependency_links to PyAdder.egg-info/dependency_links.txt
writing top-level names to PyAdder.egg-info/top_level.txt
reading manifest file 'PyAdder.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'PyAdder.egg-info/SOURCES.txt'
running build_ext
============================= test session starts ==============================
platform darwin -- Python 3.6.4, pytest-3.3.2, py-1.5.2, pluggy-0.6.0 -- /Users/jbloom/anaconda3/bin/python
cachedir: .cache
rootdir: /Users/jbloom/Classes/python-seminar/DataFiles_and_Notebooks/09_Web/PyAdder, inifile: setup.cfg
plugins: cov-2.5.1
collected 4 items                                                              

adder/tests/test_one_number.py::TestOneNumber::test_deplorables PASSED   [ 25%]
adder/tests/test_one_number.py::TestOneNumber::test_floats PASSED        [ 50%]
adder/tests/test_one_number.py::TestOneNumber::test_ints PASSED          [ 

In [17]:
!git commit adder/tests/test_one_number.py -m "added testing two numbers"

[master 3e918bc] added testing two numbers
 1 file changed, 3 insertions(+)


In [18]:
!git push

Counting objects: 5, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 504 bytes | 504.00 KiB/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/profjsb/PyAdder
   05811d6..3e918bc  master -> master


In [15]:
%%writefile adder/tests/test_one_number.py
from unittest import TestCase
import math
from numpy import inf, isinf, nan, isnan
import adder

class TestOneNumber(TestCase):

    def test_floats(self):
        for num in [1617161771.7650001, math.pi, math.pi**100,
                    math.pi**-100, 3.1]:
            self.assertEqual(adder.run(num), num)

    def test_ints(self):
        for num in [-1,0,1]:
            self.assertEqual(adder.run(num), num)

    def test_deplorables(self):
        self.assertTrue(isinf(adder.run(inf)))
        self.assertTrue(isnan(adder.run(nan)))

class TestTwoNumbers(TestCase):

    def test_floats(self):
        self.assertEqual(adder.run([2.0,3.0]), 5.0)


Overwriting adder/tests/test_one_number.py


# Continuous Integration

"Continuous Integration is a software development practice where members of a team integrate their work frequently, usually each person integrates at least daily - leading to multiple integrations per day. Each integration is verified by an automated build (including test) to detect integration errors as quickly as possible. Many teams find that this approach leads to significantly reduced integration problems and allows a team to develop cohesive software more rapidly."

http://docs.python-guide.org/en/latest/scenarios/ci/

**Travis-CI** is a distributed CI server which builds tests for open source projects for free. It provides multiple workers to run Python tests on and seamlessly integrates with GitHub. You can even have it comment on your Pull Requests whether this particular changeset breaks the build or not. So if you are hosting your code on GitHub, travis-ci is a great and easy way to get started with Continuous Integration. See https://docs.travis-ci.com/

If we're maintaining a repo, ideally we'd like to know if our tests are passing before accepting pull requests.

Let's look at the Travis CI interface: https://travis-ci.org/profjsb/PyAdder

<img src="https://www.evernote.com/l/AUUiuc2SSGNHk63FpxmZrYb2w4nSuzUry9UB/image.png">

We can add 
```html
<img src="https://travis-ci.org/profjsb/PyAdder.svg?branch=master" data-pin-nopin="true">
```

to our `README.md` to get a green badge:

<img src="https://travis-ci.org/profjsb/PyAdder.svg?branch=master" data-pin-nopin="true">

## code coverage

When tests are run, we can see what part of our code is touched ("covered").

In [44]:
!pip install pytest-cov

In [24]:
!py.test --cov-report html --cov-report term-missing --cov=./

============================= test session starts ==============================
platform darwin -- Python 3.6.4, pytest-3.3.2, py-1.5.2, pluggy-0.6.0 -- /Users/jbloom/anaconda3/bin/python
cachedir: .cache
rootdir: /Users/jbloom/Classes/python-seminar/DataFiles_and_Notebooks/09_Web/PyAdder, inifile: setup.cfg
plugins: cov-2.5.1
collected 4 items                                                              

adder/tests/test_one_number.py::TestOneNumber::test_deplorables PASSED   [ 25%]
adder/tests/test_one_number.py::TestOneNumber::test_floats PASSED        [ 50%]
adder/tests/test_one_number.py::TestOneNumber::test_ints PASSED          [ 75%]
adder/tests/test_one_number.py::TestTwoNumbers::test_floats PASSED       [100%]

---------- coverage: platform darwin, python 3.6.4-final-0 -----------
Name                Stmts   Miss Branch BrPart  Cover   Missing
---------------------------------------------------------------
adder/__init__.py       7      1      0      0    86%   10
Coverage H

In [25]:
!open htmlcov/index.html

We can keep track of code coverage using codecov and Travis (https://docs.codecov.io/docs/about-code-coverage)

see https://codecov.io/gh/profjsb/PyAdder

# Making Code Citeable

http://ivory.idyll.org/blog/2016-using-zenodo-to-archive-github.html

https://guides.github.com/activities/citable-code/

<img src="https://www.evernote.com/l/AUVFhdB6uhFC7IAz3uSz5K-L74xYniPLyQUB/image.png">
